In [1]:
#!/usr/bin/python

# import sys
import pickle
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math
# from scipy import stats
from functools import partial

# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_selection import SelectPercentile, SelectFromModel, \
        f_classif, mutual_info_classif, chi2, SelectFpr, SelectFdr, RFECV
from sklearn.decomposition import FastICA, IncrementalPCA, KernelPCA, PCA, \
                                  TruncatedSVD
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

from time import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# sys.path.append("../tools/")
# from feature_format import featureFormat, targetFeatureSplit
# from dos2unix import crlf_to_lf
# from tester import dump_classifier_and_data

In [2]:
### Load data sets and feature lists.
X_train = None
with open(file='X_train.pkl', mode='rb') as file:
    X_train = pickle.load(file=file)
y_train = None
with open(file='y_train.pkl', mode='rb') as file:
    y_train = pickle.load(file=file)
X_test = None
with open(file='X_test.pkl', mode='rb') as file:
    X_test = pickle.load(file=file)
y_test = None
with open(file='y_test.pkl', mode='rb') as file:
    y_test = pickle.load(file=file)
X_train_scaled_imp0 = None
with open('X_train_scaled_imp0.pkl', 'rb') as file:
    X_train_scaled_imp0 = pickle.load(file=file)
X_test_scaled_imp0 = None
with open('X_test_scaled_imp0.pkl', 'rb') as file:
    X_test_scaled_imp0 = pickle.load(file=file)
X_train_scaled_imp_med = None
with open('X_train_scaled_imp_med.pkl', 'rb') as file:
    X_train_scaled_imp_med = pickle.load(file=file)
X_test_scaled_imp_med = None
with open('X_test_scaled_imp_med.pkl', 'rb') as file:
    X_test_scaled_imp_med = pickle.load(file=file)
X_train_scaled_imp_mv = None
with open('X_train_scaled_imp_mv.pkl', 'rb') as file:
    X_train_scaled_imp_mv = pickle.load(file=file)
X_test_scaled_imp_mv = None
with open('X_test_scaled_imp_mv.pkl', 'rb') as file:
    X_test_scaled_imp_mv = pickle.load(file=file)


fin_features = None
with open('fin_features.pkl', 'rb') as file:
    fin_features = pickle.load(file=file)
pay_features = None
with open('pay_features.pkl', 'rb') as file:
    pay_features = pickle.load(file=file)
stock_features = None
with open('stock_features.pkl', 'rb') as file:
    stock_features = pickle.load(file=file)
email_features = None
with open('email_features.pkl', 'rb') as file:
    email_features = pickle.load(file=file)
features_list = None
with open('features_list.pkl', 'rb') as file:
    features_list = pickle.load(file=file)
pay_feats_divby_lst = None
with open('pay_feats_divby_lst.pkl', 'rb') as file:
    pay_feats_divby_lst = pickle.load(file=file)
stock_feats_divby_lst=None
with open('stock_feats_divby_lst.pkl', 'rb') as file:
    stock_feats_divby_lst = pickle.load(file=file)
email_feats_divby_lst = None
with open('email_feats_divby_lst.pkl', 'rb') as file:
    email_feats_divby_lst = pickle.load(file=file)
quant_flags_lst = None
with open('quant_flags_lst.pkl', 'rb') as file:
    quant_flags_lst = pickle.load(file=file)
sign_flags_lst = None
with open('sign_flags_lst.pkl', 'rb') as file:
    sign_flags_lst = pickle.load(file=file)

In [3]:
X_train.dropna(inplace=True)
y_train = y_train.loc[X_train.index]
X_train.info()
selector = SelectPercentile(score_func=chi2).fit(X=X_train, y=y_train)
X_train = selector.transform(X=X_train)
X_train.shape

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, KAMINSKI WINCENTY J to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: float64(235), object(47)
memory usage: 39.8+ KB


(18, 29)

In [3]:
### Oops, forgot to keep these in a list.
boost_feats_lst = ['lin_reg', 'k_means']
features_list = features_list + boost_feats_lst

In [9]:
### And, I forgot to scale lin_reg
scaler = MinMaxScaler()
X_train_scaled_imp0['lin_reg'] = scaler.fit_transform(\
                    X=X_train_scaled_imp0['lin_reg'].values.reshape(-1, 1))
X_test_scaled_imp0['lin_reg'] = scaler.transform(\
                    X=X_test_scaled_imp0['lin_reg'].values.reshape(-1, 1))
X_train_scaled_imp_med['lin_reg'] = scaler.fit_transform(\
                    X=X_train_scaled_imp_med['lin_reg'].values.reshape(-1, 1))
X_test_scaled_imp_med['lin_reg'] = scaler.transform(\
                    X=X_test_scaled_imp_med['lin_reg'].values.reshape(-1, 1))
X_train_scaled_imp_mv['lin_reg'] = scaler.fit_transform(\
                    X=X_train_scaled_imp_mv['lin_reg'].values.reshape(-1, 1))
X_test_scaled_imp_mv['lin_reg'] = scaler.transform(\
                    X=X_test_scaled_imp_mv['lin_reg'].values.reshape(-1, 1))

In [12]:
X_train_scaled_imp0['lin_reg']

ECHOLS JOHN B          4.184153e-15
KAMINSKI WINCENTY J    2.629841e-15
BOWEN JR RAYMOND M     1.000000e+00
PIPER GREGORY F        4.274359e-15
HAYES ROBERT E         3.212708e-15
                           ...     
HIRKO JOSEPH           1.000000e+00
GIBBS DANA R           5.322132e-15
SHARP VICTORIA T       2.851885e-15
PIRO JIM               3.705369e-15
MURRAY JULIA H         4.767020e-15
Name: lin_reg, Length: 86, dtype: float64

In [13]:
### And, I forgot to dink around with my boost features.
def look_at_lin_reg(X_train, y_train, X_test, y_test, title):
    print(title)
    df = pd.concat(objs=[X_train['lin_reg'], y_train], axis=1)
    print('Train:')
    print('POIs:')
    print(df.loc[df['poi'] == 1])
    print('Predicted POIs:')
    print(df.loc[abs((df['lin_reg'] - 1)) < .0001])
    
    print('Test:')
    df = pd.concat(objs=[X_test['lin_reg'], y_test], axis=1)
    print('POIs:')
    print(df.loc[df['poi'] == 1])
    print('Predicted POIs:')
    print(df.loc[abs((df['lin_reg'] - 1)) < 1])
    
    return

look_at_lin_reg(X_train=X_train_scaled_imp0, y_train=y_train,
                X_test=X_test_scaled_imp0, y_test=y_test, title='imp0')
look_at_lin_reg(X_train=X_train_scaled_imp_med, y_train=y_train,
                X_test=X_test_scaled_imp_med, y_test=y_test, title='imp_med')
look_at_lin_reg(X_train=X_train_scaled_imp_mv, y_train=y_train,
                X_test=X_test_scaled_imp_mv, y_test=y_test, title='imp_mv')

imp0
Train:
POIs:
                    lin_reg  poi
BOWEN JR RAYMOND M      1.0    1
BELDEN TIMOTHY N        1.0    1
KOPPER MICHAEL J        1.0    1
RIEKER PAULA H          1.0    1
YEAGER F SCOTT          1.0    1
CAUSEY RICHARD A        1.0    1
KOENIG MARK E           1.0    1
DELAINEY DAVID W        1.0    1
FASTOW ANDREW S         1.0    1
SKILLING JEFFREY K      1.0    1
GLISAN JR BEN F         1.0    1
HIRKO JOSEPH            1.0    1
Predicted POIs:
                    lin_reg  poi
BOWEN JR RAYMOND M      1.0    1
BELDEN TIMOTHY N        1.0    1
KOPPER MICHAEL J        1.0    1
RIEKER PAULA H          1.0    1
YEAGER F SCOTT          1.0    1
CAUSEY RICHARD A        1.0    1
KOENIG MARK E           1.0    1
DELAINEY DAVID W        1.0    1
FASTOW ANDREW S         1.0    1
SKILLING JEFFREY K      1.0    1
GLISAN JR BEN F         1.0    1
HIRKO JOSEPH            1.0    1
Test:
POIs:
                       lin_reg  poi
SHELBY REX           -0.362908    1
HANNON KEVIN P       -0.

In [14]:
def look_at_k_means(X_train, y_train, X_test, y_test, title):
    print(title)
    print('Train:')
    df = pd.concat(objs=[X_train['k_means'], y_train], axis=1)
    print(pd.concat(objs=[\
                (df.groupby('k_means').sum() / df.groupby('k_means').count()),
                 df.groupby('k_means').count()], axis=1))
    print('Test:')
    df = pd.concat(objs=[X_test['k_means'], y_test], axis=1)
    print(pd.concat(objs=[\
                (df.groupby('k_means').sum() / df.groupby('k_means').count()),
                 df.groupby('k_means').count()], axis=1))
    
    return

look_at_k_means(X_train=X_train_scaled_imp0, y_train=y_train,
                X_test=X_test_scaled_imp0, y_test=y_test, title='imp0')
look_at_k_means(X_train=X_train_scaled_imp_med, y_train=y_train,
                X_test=X_test_scaled_imp_med, y_test=y_test, title='imp_med')
look_at_k_means(X_train=X_train_scaled_imp_mv, y_train=y_train,
                X_test=X_test_scaled_imp_mv, y_test=y_test, title='imp_mv')

imp0
Train:
              poi  poi
k_means               
0        0.166667   18
1        0.066667   15
2        0.150000   20
3        0.000000   12
4        0.238095   21
Test:
              poi  poi
k_means               
0        0.166667    6
1        0.000000    6
2        0.000000    7
3        0.000000    2
4        0.312500   16
imp_med
Train:
              poi  poi
k_means               
0        0.181818   11
1        0.272727   11
2        0.187500   16
3        0.111111   36
4        0.000000   12
Test:
              poi  poi
k_means               
0        0.000000    2
1        0.428571    7
2        0.200000   10
3        0.062500   16
4        0.000000    2
imp_mv
Train:
              poi  poi
k_means               
0        0.307692   26
1        0.000000    9
2        0.000000   18
3        0.000000   10
4        0.173913   23
Test:
              poi  poi
k_means               
0        0.250000   12
1        0.000000    2
2        0.230769   13
4        0.000000   1

### Get a baseline with all features

In [16]:
def run_skl(method, X_train, y_train, X_test, y_test, print_perf=True,
            perf_series='', **kwargs):
    '''Train and test sklearn supervised ML models.
    ### Needs `from class_vis import prettyPicture` to graph.
    Parameters:
        method: callable sklearn supervised ML model.
        X_train: training feature matrix.
        y_train: training label 1D array. Series may throw warning.
        X_test: testing feature matrix.
        y_test: testing label 1D array. Series may throw warning.
        print_perf: {bool} True (default) prints performance metrics.
        per_series: {str} (default is empty) If not empty,
                    returns pandas Series with model and performance metrics.
                    String value is name of series.
        **kwargs: arguments to pass to method.
    Returns:
        clf: trained model.
        pred: array of predictions.
        conf: confusion matrix of test results.
        prf: sklearn.metrics.precision_recall_fscore_support results, weighted.
        perf_sr: pandas Series of model and metrics (everything but pred).
                 Must be binary class. Returns as None if perf_series False.
    '''
    clf = None
    pred = None
    conf = None
    prf = None
    perf_sr = None
    
    clf = method(**kwargs)

    t0 = time()
    clf = clf.fit(X=X_train, y=y_train)
    train_t = round(time()-t0, 3)
    
    t0 = time()
    pred = clf.predict(X=X_test)
    pred_t = round(time()-t0, 3)
    
    conf = confusion_matrix(y_true=y_test, y_pred=pred)
    prf = precision_recall_fscore_support(y_true=y_test, y_pred=pred)
    
###
### TO DO: fix from class_vis import prettyPicture
###
#     try:
#         prettyPicture(clf, X_test, y_test)
#     except NameError:
#         pass
    
    if print_perf:
        print(clf)
        print("Training time:", train_t, "s")
        print("Prediction time:", pred_t, "s")
        print("Confusion matrix:\n", conf)
        print("Precision, recall, f beta score, support:\n", prf)
    
    if perf_series != '':
        perf_sr = pd.Series(
            {
                'model': clf,
                'nonPOI_prec': prf[0][0],
                'POI_prec': prf[0][1],
                'nonPOI_rec': prf[1][0],
                'POI_rec': prf[1][1],
                'nonPOI_f': prf[2][0],
                'POI_f': prf[2][1],
                'nonPOI_sup': prf[3][0],
                'POI_sup': prf[3][1],
                't_neg': conf[0][0],
                'f_neg': conf[0][1],
                'f_pos': conf[1][0],
                't_pos': conf[1][1],
                'train_t': train_t,
                'predict_t': pred_t,
            },
            name=perf_series
        )
    
    return clf, pred, conf, prf, perf_sr

ordered_cols_lst = ['nonPOI_prec', 'POI_prec', 'nonPOI_rec', 'POI_rec',
                    'nonPOI_f', 'POI_f', 'nonPOI_sup', 'POI_sup', 't_neg',
                    'f_neg', 'f_pos', 't_pos', 'train_t', 'predict_t', 'model']

y_train_1d = np.ravel(y_train)
y_test_1d = np.ravel(y_test)

base_perf_imp0_df = pd.DataFrame(columns=ordered_cols_lst)
base_perf_imp_med_df = pd.DataFrame(columns=ordered_cols_lst)
base_perf_imp_mv_df = pd.DataFrame(columns=ordered_cols_lst)

base_perfs_dict = {'imp0': base_perf_imp0_df,
                  'imp_med': base_perf_imp_med_df,
                  'imp_mv': base_perf_imp_mv_df}
imp_sets_dict = {'imp0': [X_train_scaled_imp0, X_test_scaled_imp0],
                 'imp_med': [X_train_scaled_imp_med, X_test_scaled_imp_med],
                 'imp_mv': [X_train_scaled_imp_mv, X_test_scaled_imp_mv]}
clf_dict = {'dt_clf': DecisionTreeClassifier, 'rf_clf': RandomForestClassifier,
            'ab_clf': AdaBoostClassifier, 'kn_clf': KNeighborsClassifier,
            'gnb_clf': GaussianNB, 'svc_clf': svm.SVC}


def get_base_perfs(base_perfs_dict, imp_sets_dict, clf_dict, X_test, y_test):
    
#     copy_base_perfs_dict = base_perfs_dict.deepcopy() #dict has no deepcopy
    
    for imp_name, imp_sets in imp_sets_dict.items():
        print('\n', imp_name)
        for clf_name, method in clf_dict.items():
            print('\n', clf_name)
            _, _, _, _, perf_sr = run_skl(method=method,
                                          X_train=imp_sets[0], y_train=y_train,
                                          X_test=imp_sets[1], y_test=y_test,
                                          perf_series=clf_name)
            base_perfs_dict[imp_name] \
                = base_perfs_dict[imp_name].append(perf_sr)
    
    return #copy_base_perfs_dict


get_base_perfs(base_perfs_dict=base_perfs_dict,
                                 imp_sets_dict=imp_sets_dict,
                                 clf_dict=clf_dict, X_test=X_test,
                                 y_test=y_test)

first_base_df = None
with open('base_perf_df.pkl', 'rb') as file:
    first_base_df = pickle.load(file=file)
    
base_perfs_dict['first_base'] = first_base_df


 imp0

 dt_clf
DecisionTreeClassifier()
Training time: 0.004 s
Prediction time: 0.003 s
Confusion matrix:
 [[25  6]
 [ 5  1]]
Precision, recall, f beta score, support:
 (array([0.83333333, 0.14285714]), array([0.80645161, 0.16666667]), array([0.81967213, 0.15384615]), array([31,  6], dtype=int64))

 rf_clf
RandomForestClassifier()
Training time: 0.101 s
Prediction time: 0.0 s
Confusion matrix:
 [[30  1]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.90909091, 0.75      ]), array([0.96774194, 0.5       ]), array([0.9375, 0.6   ]), array([31,  6], dtype=int64))

 ab_clf
AdaBoostClassifier()
Training time: 0.0 s
Prediction time: 0.0 s
Confusion matrix:
 [[25  6]
 [ 5  1]]
Precision, recall, f beta score, support:
 (array([0.83333333, 0.14285714]), array([0.80645161, 0.16666667]), array([0.81967213, 0.15384615]), array([31,  6], dtype=int64))

 kn_clf
KNeighborsClassifier()
Training time: 0.016 s
Prediction time: 0.0 s
Confusion matrix:
 [[29  2]
 [ 6  0]]
Precision, recal

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier()
Training time: 0.103 s
Prediction time: 0.016 s
Confusion matrix:
 [[29  2]
 [ 4  2]]
Precision, recall, f beta score, support:
 (array([0.87878788, 0.5       ]), array([0.93548387, 0.33333333]), array([0.90625, 0.4    ]), array([31,  6], dtype=int64))

 ab_clf
AdaBoostClassifier()
Training time: 0.0 s
Prediction time: 0.016 s
Confusion matrix:
 [[27  4]
 [ 6  0]]
Precision, recall, f beta score, support:
 (array([0.81818182, 0.        ]), array([0.87096774, 0.        ]), array([0.84375, 0.     ]), array([31,  6], dtype=int64))

 kn_clf
KNeighborsClassifier()
Training time: 0.0 s
Prediction time: 0.016 s
Confusion matrix:
 [[29  2]
 [ 6  0]]
Precision, recall, f beta score, support:
 (array([0.82857143, 0.        ]), array([0.93548387, 0.        ]), array([0.87878788, 0.        ]), array([31,  6], dtype=int64))

 gnb_clf
GaussianNB()
Training time: 0.0 s
Prediction time: 0.0 s
Confusion matrix:
 [[29  2]
 [ 6  0]]
Precision, recall, f beta score, support:
 (arr

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
key_order_lst = ['first_base', 'imp0', 'imp_med', 'imp_mv']
for key in key_order_lst:
    print('\n', key)
    base_perfs_dict[key]


 first_base


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.875000,0.40,0.903226,0.333333,0.888889,0.363636,31,6,28,3,4,2,0.000,0.000,DecisionTreeClassifier()
rf_clf,0.885714,1.00,1.000000,0.333333,0.939394,0.500000,31,6,31,0,4,2,0.096,0.003,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.875000,0.40,0.903226,0.333333,0.888889,0.363636,31,6,28,3,4,2,0.047,0.000,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.861111,1.00,1.000000,0.166667,0.925373,0.285714,31,6,31,0,5,1,0.000,0.000,KNeighborsClassifier()
gnb_clf,0.833333,0.16,0.322581,0.666667,0.465116,0.258065,31,6,10,21,2,4,0.000,0.000,GaussianNB()
svc_clf,0.837838,0.00,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.005,0.002,SVC()



 imp0


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.833333,0.142857,0.806452,0.166667,0.819672,0.153846,31,6,25,6,5,1,0.004,0.003,DecisionTreeClassifier()
rf_clf,0.909091,0.750000,0.967742,0.500000,0.937500,0.600000,31,6,30,1,3,3,0.101,0.000,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.833333,0.142857,0.806452,0.166667,0.819672,0.153846,31,6,25,6,5,1,0.000,0.000,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.828571,0.000000,0.935484,0.000000,0.878788,0.000000,31,6,29,2,6,0,0.016,0.000,KNeighborsClassifier()
gnb_clf,0.848485,0.250000,0.903226,0.166667,0.875000,0.200000,31,6,28,3,5,1,0.016,0.000,GaussianNB()
svc_clf,0.837838,0.000000,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.000,0.000,SVC()



 imp_med


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.818182,0.0,0.870968,0.000000,0.843750,0.0,31,6,27,4,6,0,0.000,0.000,DecisionTreeClassifier()
rf_clf,0.878788,0.5,0.935484,0.333333,0.906250,0.4,31,6,29,2,4,2,0.103,0.016,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.818182,0.0,0.870968,0.000000,0.843750,0.0,31,6,27,4,6,0,0.000,0.016,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.828571,0.0,0.935484,0.000000,0.878788,0.0,31,6,29,2,6,0,0.000,0.016,KNeighborsClassifier()
gnb_clf,0.828571,0.0,0.935484,0.000000,0.878788,0.0,31,6,29,2,6,0,0.000,0.000,GaussianNB()
svc_clf,0.837838,0.0,1.000000,0.000000,0.911765,0.0,31,6,31,0,6,0,0.000,0.000,SVC()



 imp_mv


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.900000,0.428571,0.870968,0.500000,0.885246,0.461538,31,6,27,4,3,3,0.016,0.000,DecisionTreeClassifier()
rf_clf,0.882353,0.666667,0.967742,0.333333,0.923077,0.444444,31,6,30,1,4,2,0.108,0.000,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.900000,0.428571,0.870968,0.500000,0.885246,0.461538,31,6,27,4,3,3,0.016,0.007,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.852941,0.333333,0.935484,0.166667,0.892308,0.222222,31,6,29,2,5,1,0.004,0.006,KNeighborsClassifier()
gnb_clf,0.852941,0.333333,0.935484,0.166667,0.892308,0.222222,31,6,29,2,5,1,0.004,0.002,GaussianNB()
svc_clf,0.837838,0.000000,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.005,0.003,SVC()


### Feature selection

In [18]:
### First, look for suspiciously highly significant features.
clf_imp0 = LinearRegression().fit(X=X_train_scaled_imp0, y=y_train)
clf_imp_med = LinearRegression().fit(X=X_train_scaled_imp_med, y=y_train)
clf_imp_mv = LinearRegression().fit(X=X_train_scaled_imp_mv, y=y_train)

coefs_df = pd.DataFrame(data={'clf_imp0.coef_': clf_imp0.coef_,
                              'clf_imp_med.coef_': clf_imp_med.coef_,
                              'clf_imp_mv.coef_': clf_imp_mv.coef_},
                        index=X_train_scaled_imp0.columns)
coefs_df

imp0_sort = np.sort(clf_imp0.coef_)
impmed_sort = np.sort(clf_imp_med.coef_)
impmv_sort = np.sort(clf_imp_mv.coef_)
high_idx = -4
low_idx = 3

coefs_df.loc[(coefs_df['clf_imp0.coef_'] >= imp0_sort[high_idx])\
             | (coefs_df['clf_imp0.coef_'] <= imp0_sort[low_idx])]\
    ['clf_imp0.coef_'].sort_values()
coefs_df.loc[(coefs_df['clf_imp_med.coef_'] >= impmed_sort[high_idx])\
             | (coefs_df['clf_imp_med.coef_'] <= impmed_sort[low_idx])]\
    ['clf_imp_med.coef_'].sort_values()
coefs_df.loc[(coefs_df['clf_imp_mv.coef_'] >= impmv_sort[high_idx])\
             | (coefs_df['clf_imp_mv.coef_'] <= impmv_sort[low_idx])]\
    ['clf_imp_mv.coef_'].sort_values()

,clf_imp0.coef_,clf_imp_med.coef_,clf_imp_mv.coef_
salary,-0.006147,-0.000197,-0.003670
bonus,0.030807,0.014358,0.001248
long_term_incentive,0.005510,-0.011974,-0.009641
expenses,0.005437,0.005198,0.002662
total_payments,-0.021599,-0.015705,-0.011992
...,...,...,...
shared_receipt_with_poi_DivBy_from_poi_to_this_person_sign,0.000000,0.000000,0.000000
from_messages_DivBy_from_this_person_to_poi_sign,0.000000,0.000000,0.000000
from_this_person_to_poi_DivBy_from_messages_sign,-0.004089,-0.014521,-0.007546
lin_reg,0.918067,0.930533,0.958049


expenses_DivBy_salary_outliers                          -0.075803
from_poi_to_this_person_DivBy_shared_receipt_with_poi   -0.048748
restricted_stock                                        -0.045761
shared_receipt_with_poi_q_4                             -0.041778
exercised_stock_options                                  0.040414
total_stock_value_q_4                                    0.054755
from_poi_to_this_person_q_5                              0.058005
lin_reg                                                  0.918067
Name: clf_imp0.coef_, dtype: float64

expenses_DivBy_salary_outliers                -0.070489
salary_DivBy_expenses_q_5                     -0.036609
restricted_stock                              -0.036009
shared_receipt_with_poi_q_4                   -0.032378
expenses_DivBy_long_term_incentive_outliers    0.038483
expenses_DivBy_long_term_incentive             0.042131
salary_DivBy_expenses_q_4                      0.045963
lin_reg                                        0.930533
Name: clf_imp_med.coef_, dtype: float64

from_messages                                               -0.035927
expenses_DivBy_salary_outliers                              -0.031566
total_payments_DivBy_salary_q_5                             -0.030747
expenses_q_5                                                -0.030696
total_payments_DivBy_expenses_q_4                            0.032486
from_poi_to_this_person_DivBy_shared_receipt_with_poi_q_5    0.034939
expenses_DivBy_bonus_q_4                                     0.036575
lin_reg                                                      0.958049
Name: clf_imp_mv.coef_, dtype: float64

In [19]:
### First, look for suspiciously highly significant features.
clf_imp0 = DecisionTreeClassifier().fit(X=X_train_scaled_imp0, y=y_train)
clf_imp_med = DecisionTreeClassifier().fit(X=X_train_scaled_imp_med, y=y_train)
clf_imp_mv = DecisionTreeClassifier().fit(X=X_train_scaled_imp_mv, y=y_train)
coefs_df = pd.DataFrame(data={
    'clf_imp0.feature_importances_': clf_imp0.feature_importances_,
    'clf_imp_med.feature_importances_': clf_imp_med.feature_importances_,
    'clf_imp_mv.feature_importances_': clf_imp_mv.feature_importances_
},
                        index=X_train_scaled_imp0.columns)
coefs_df

coefs_df.loc[(coefs_df['clf_imp0.feature_importances_'] > 0)]\
    ['clf_imp0.feature_importances_']
coefs_df.loc[(coefs_df['clf_imp_med.feature_importances_'] > 0)]\
    ['clf_imp_med.feature_importances_']
coefs_df.loc[(coefs_df['clf_imp_mv.feature_importances_'] > 0)]\
    ['clf_imp_mv.feature_importances_']

,clf_imp0.feature_importances_,clf_imp_med.feature_importances_,clf_imp_mv.feature_importances_
salary,0.0,0.0,0.0
bonus,0.0,0.0,0.0
long_term_incentive,0.0,0.0,0.0
expenses,0.0,0.0,0.0
total_payments,0.0,0.0,0.0
...,...,...,...
shared_receipt_with_poi_DivBy_from_poi_to_this_person_sign,0.0,0.0,0.0
from_messages_DivBy_from_this_person_to_poi_sign,0.0,0.0,0.0
from_this_person_to_poi_DivBy_from_messages_sign,0.0,0.0,0.0
lin_reg,1.0,1.0,1.0


lin_reg    1.0
Name: clf_imp0.feature_importances_, dtype: float64

lin_reg    1.0
Name: clf_imp_med.feature_importances_, dtype: float64

lin_reg    1.0
Name: clf_imp_mv.feature_importances_, dtype: float64

lin_reg definitely overshadows the rest of the features, but there was no leakage when creating it.

Its score is perfect, which is to be expected since its only on the training data. Also, this smacks of overfitting. I'm going to drop it, and the k_means cluster feature based on it. I'll create the k_means cluster again, though.

In [20]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_pred=X_train_scaled_imp0['lin_reg'],
                         y_true=y_train)
explained_variance_score(y_pred=X_train_scaled_imp_med['lin_reg'],
                         y_true=y_train)
explained_variance_score(y_pred=X_train_scaled_imp_mv['lin_reg'],
                         y_true=y_train)

1.0

1.0

1.0

In [21]:
### Remove lin_reg
boost_feats_lst.remove('lin_reg')
features_list.remove('lin_reg')
X_train_scaled_imp0.drop('lin_reg', axis=1, inplace=True)
X_train_scaled_imp_med.drop('lin_reg', axis=1,  inplace=True)
X_train_scaled_imp_mv.drop('lin_reg', axis=1, inplace=True)
X_test_scaled_imp0.drop('lin_reg', axis=1, inplace=True)
X_test_scaled_imp_med.drop('lin_reg', axis=1,  inplace=True)
X_test_scaled_imp_mv.drop('lin_reg', axis=1, inplace=True)

### Reconstruct Kmeans
n_clusters = 5
clf_imp0 = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp0['k_means'] = clf_imp0.fit_predict(X=X_train_scaled_imp0)
X_test_scaled_imp0['k_means'] = clf_imp0.predict(X=X_test_scaled_imp0)

clf_imp_med = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp_med['k_means'] = clf_imp_med.fit_predict(X=X_train_scaled_imp_med)
X_test_scaled_imp_med['k_means'] = clf_imp_med.predict(X=X_test_scaled_imp_med)

clf_imp_mv = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp_mv['k_means'] = clf_imp_mv.fit_predict(X=X_train_scaled_imp_mv)
X_test_scaled_imp_mv['k_means'] = clf_imp_mv.predict(X=X_test_scaled_imp_mv)

In [22]:
look_at_k_means(X_train=X_train_scaled_imp0, y_train=y_train,
                X_test=X_test_scaled_imp0, y_test=y_test, title='imp0')
look_at_k_means(X_train=X_train_scaled_imp_med, y_train=y_train,
                X_test=X_test_scaled_imp_med, y_test=y_test, title='imp_med')
look_at_k_means(X_train=X_train_scaled_imp_mv, y_train=y_train,
                X_test=X_test_scaled_imp_mv, y_test=y_test, title='imp_mv')

imp0
Train:
              poi  poi
k_means               
0        0.066667   15
1        0.238095   21
2        0.000000   12
3        0.150000   20
4        0.166667   18
Test:
              poi  poi
k_means               
0        0.000000    6
1        0.312500   16
2        0.000000    2
3        0.000000    7
4        0.166667    6
imp_med
Train:
              poi  poi
k_means               
0        0.100000   40
1        0.181818   11
2        0.000000   12
3        0.200000   10
4        0.307692   13
Test:
              poi  poi
k_means               
0        0.055556   18
1        0.000000    2
2        0.000000    2
3        0.600000    5
4        0.200000   10
imp_mv
Train:
              poi  poi
k_means               
0        0.000000   18
1        0.000000    9
2        0.173913   23
3        0.307692   26
4        0.000000   10
Test:
              poi  poi
k_means               
0        0.230769   13
1        0.000000    2
2        0.000000   10
3        0.250000   1

Actually, I'll just make several n_clusters k_means features.

In [23]:
### Remove k_means
boost_feats_lst.remove('k_means')
features_list.remove('k_means')
X_train_scaled_imp0.drop('k_means', axis=1, inplace=True)
X_train_scaled_imp_med.drop('k_means', axis=1,  inplace=True)
X_train_scaled_imp_mv.drop('k_means', axis=1, inplace=True)
X_test_scaled_imp0.drop('k_means', axis=1, inplace=True)
X_test_scaled_imp_med.drop('k_means', axis=1,  inplace=True)
X_test_scaled_imp_mv.drop('k_means', axis=1, inplace=True)

In [24]:
np.arange(start=1, stop=2, step=1)

array([1])

In [25]:
def add_k_means_n(X_train, X_test, n_cluster_lst=[2, 3, 4, 5, 6, 7, 8, 9, 10]):
    '''For each n_cluster, create n features that flag cluster membership.
    Parameters:
        X_train: (pandas.DataFrame) training set to fit and add features to.
        X_test: (pandas.DataFrame) testing set to add features to.
        n_cluster_lst: (list, int) n clusters to classify.
    Returns:
        train_cluster_subspace: (pandas.DataFrame, bool) new features to add to
            training set.
        test_cluster_subspace: (pandas.DataFrame, bool) new features to add to
            testing set.'''
    
    train_cluster_subspace = pd.DataFrame(index=X_train.index)
    test_cluster_subspace = pd.DataFrame(index=X_test.index)
    
    lamb = lambda x: x == i
    
    for n in n_cluster_lst:
        clf = KMeans(n_clusters=n, random_state=42).fit(X=X_train)
        train_sr = pd.Series(data=clf.predict(X=X_train),
                             index=X_train.index)
#         print('\ntrain_sr:\n', train_sr)
        test_sr = pd.Series(data=clf.predict(X=X_test),
                            index=X_test.index)
#         print('\ntest_sr\n:', test_sr)
        
        for i in np.arange(start=0, stop=n, step=1):
            col_name = 'k_means_' + str(n) + '_n_' + str(i)
            
            flag_sr = train_sr.apply(func=lamb)
#             flag_sr.index = X_train.index
            flag_sr.name = col_name
#             print('flag_sr.name:' + flag_sr.name)
            train_cluster_subspace = pd.concat(objs=[train_cluster_subspace,
                                                     flag_sr], axis=1)
            
            flag_sr = test_sr.apply(func=lamb)
#             flag_sr.index = X_test.index
            flag_sr.name = col_name
#             print('flag_sr.name:' + flag_sr.name)
            test_cluster_subspace = pd.concat(objs=[test_cluster_subspace,
                                                    flag_sr], axis=1)
    
    return train_cluster_subspace, test_cluster_subspace
    
train_cluster_subspace, test_cluster_subspace = \
    add_k_means_n(X_train=X_train_scaled_imp0, X_test=X_test_scaled_imp0)

train_cluster_subspace.info()
train_cluster_subspace.head()
test_cluster_subspace.info()
test_cluster_subspace.head()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   k_means_2_n_0   86 non-null     bool 
 1   k_means_2_n_1   86 non-null     bool 
 2   k_means_3_n_0   86 non-null     bool 
 3   k_means_3_n_1   86 non-null     bool 
 4   k_means_3_n_2   86 non-null     bool 
 5   k_means_4_n_0   86 non-null     bool 
 6   k_means_4_n_1   86 non-null     bool 
 7   k_means_4_n_2   86 non-null     bool 
 8   k_means_4_n_3   86 non-null     bool 
 9   k_means_5_n_0   86 non-null     bool 
 10  k_means_5_n_1   86 non-null     bool 
 11  k_means_5_n_2   86 non-null     bool 
 12  k_means_5_n_3   86 non-null     bool 
 13  k_means_5_n_4   86 non-null     bool 
 14  k_means_6_n_0   86 non-null     bool 
 15  k_means_6_n_1   86 non-null     bool 
 16  k_means_6_n_2   86 non-null     bool 
 17  k_means_6_n_3   86 non-null     bool 
 18  k_means_6_n_4

,k_means_2_n_0,k_means_2_n_1,k_means_3_n_0,k_means_3_n_1,k_means_3_n_2,k_means_4_n_0,k_means_4_n_1,k_means_4_n_2,k_means_4_n_3,k_means_5_n_0,...,k_means_10_n_0,k_means_10_n_1,k_means_10_n_2,k_means_10_n_3,k_means_10_n_4,k_means_10_n_5,k_means_10_n_6,k_means_10_n_7,k_means_10_n_8,k_means_10_n_9
ECHOLS JOHN B,False,True,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
KAMINSKI WINCENTY J,False,True,True,False,False,True,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
BOWEN JR RAYMOND M,False,True,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
PIPER GREGORY F,False,True,True,False,False,True,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
HAYES ROBERT E,True,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   k_means_2_n_0   37 non-null     bool 
 1   k_means_2_n_1   37 non-null     bool 
 2   k_means_3_n_0   37 non-null     bool 
 3   k_means_3_n_1   37 non-null     bool 
 4   k_means_3_n_2   37 non-null     bool 
 5   k_means_4_n_0   37 non-null     bool 
 6   k_means_4_n_1   37 non-null     bool 
 7   k_means_4_n_2   37 non-null     bool 
 8   k_means_4_n_3   37 non-null     bool 
 9   k_means_5_n_0   37 non-null     bool 
 10  k_means_5_n_1   37 non-null     bool 
 11  k_means_5_n_2   37 non-null     bool 
 12  k_means_5_n_3   37 non-null     bool 
 13  k_means_5_n_4   37 non-null     bool 
 14  k_means_6_n_0   37 non-null     bool 
 15  k_means_6_n_1   37 non-null     bool 
 16  k_means_6_n_2   37 non-null     bool 
 17  k_means_6_n_3   37 non-null     bool 
 18  k_means_

,k_means_2_n_0,k_means_2_n_1,k_means_3_n_0,k_means_3_n_1,k_means_3_n_2,k_means_4_n_0,k_means_4_n_1,k_means_4_n_2,k_means_4_n_3,k_means_5_n_0,...,k_means_10_n_0,k_means_10_n_1,k_means_10_n_2,k_means_10_n_3,k_means_10_n_4,k_means_10_n_5,k_means_10_n_6,k_means_10_n_7,k_means_10_n_8,k_means_10_n_9
WESTFAHL RICHARD K,True,False,False,False,True,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
SHELBY REX,False,True,True,False,False,True,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
DEFFNER JOSEPH M,False,True,True,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
WASAFF GEORGE,False,True,True,False,False,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
HANNON KEVIN P,False,True,True,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [26]:
X_train_scaled_imp0 = pd.concat(objs=[X_train_scaled_imp0,
                                      train_cluster_subspace], axis=1)
X_test_scaled_imp0 = pd.concat(objs=[X_test_scaled_imp0,
                                     test_cluster_subspace], axis=1)
X_train_scaled_imp0.info()
X_train_scaled_imp0.head()
X_test_scaled_imp0.info()
X_train_scaled_imp0.head()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 141.5+ KB


,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,k_means_10_n_0,k_means_10_n_1,k_means_10_n_2,k_means_10_n_3,k_means_10_n_4,k_means_10_n_5,k_means_10_n_6,k_means_10_n_7,k_means_10_n_8,k_means_10_n_9
ECHOLS JOHN B,0.102281,0.023508,1.000000,0.091733,0.303062,0.018630,0.042848,0.031888,0.000000,0.000000,...,False,False,False,False,False,True,False,False,False,False
KAMINSKI WINCENTY J,0.192009,0.059675,0.136531,0.363709,0.116274,0.026717,0.009365,0.030817,0.380976,0.134426,...,False,False,False,True,False,False,False,False,False,False
BOWEN JR RAYMOND M,0.195391,0.231465,0.488524,0.286230,0.300417,0.000000,0.024356,0.007263,0.150800,0.459016,...,False,False,False,False,False,False,False,True,False,False
PIPER GREGORY F,0.116627,0.059675,0.000000,0.186082,0.191991,0.027702,0.043092,0.027702,0.098886,0.200000,...,False,False,True,False,False,False,False,False,False,False
HAYES ROBERT E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012385,0.003989,0.037428,0.052459,...,False,False,False,False,False,False,False,False,False,True


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 60.0+ KB


,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,k_means_10_n_0,k_means_10_n_1,k_means_10_n_2,k_means_10_n_3,k_means_10_n_4,k_means_10_n_5,k_means_10_n_6,k_means_10_n_7,k_means_10_n_8,k_means_10_n_9
ECHOLS JOHN B,0.102281,0.023508,1.000000,0.091733,0.303062,0.018630,0.042848,0.031888,0.000000,0.000000,...,False,False,False,False,False,True,False,False,False,False
KAMINSKI WINCENTY J,0.192009,0.059675,0.136531,0.363709,0.116274,0.026717,0.009365,0.030817,0.380976,0.134426,...,False,False,False,True,False,False,False,False,False,False
BOWEN JR RAYMOND M,0.195391,0.231465,0.488524,0.286230,0.300417,0.000000,0.024356,0.007263,0.150800,0.459016,...,False,False,False,False,False,False,False,True,False,False
PIPER GREGORY F,0.116627,0.059675,0.000000,0.186082,0.191991,0.027702,0.043092,0.027702,0.098886,0.200000,...,False,False,True,False,False,False,False,False,False,False
HAYES ROBERT E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012385,0.003989,0.037428,0.052459,...,False,False,False,False,False,False,False,False,False,True


In [27]:
train_cluster_subspace, test_cluster_subspace = \
    add_k_means_n(X_train=X_train_scaled_imp_med, X_test=X_test_scaled_imp_med)
train_cluster_subspace.info()
test_cluster_subspace.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   k_means_2_n_0   86 non-null     bool 
 1   k_means_2_n_1   86 non-null     bool 
 2   k_means_3_n_0   86 non-null     bool 
 3   k_means_3_n_1   86 non-null     bool 
 4   k_means_3_n_2   86 non-null     bool 
 5   k_means_4_n_0   86 non-null     bool 
 6   k_means_4_n_1   86 non-null     bool 
 7   k_means_4_n_2   86 non-null     bool 
 8   k_means_4_n_3   86 non-null     bool 
 9   k_means_5_n_0   86 non-null     bool 
 10  k_means_5_n_1   86 non-null     bool 
 11  k_means_5_n_2   86 non-null     bool 
 12  k_means_5_n_3   86 non-null     bool 
 13  k_means_5_n_4   86 non-null     bool 
 14  k_means_6_n_0   86 non-null     bool 
 15  k_means_6_n_1   86 non-null     bool 
 16  k_means_6_n_2   86 non-null     bool 
 17  k_means_6_n_3   86 non-null     bool 
 18  k_means_6_n_4

In [28]:
X_train_scaled_imp_med = pd.concat(objs=[X_train_scaled_imp_med,
                                      train_cluster_subspace], axis=1)
X_test_scaled_imp_med = pd.concat(objs=[X_test_scaled_imp_med,
                                     test_cluster_subspace], axis=1)
X_train_scaled_imp_med.info()
X_test_scaled_imp_med.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 141.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 60.0+ KB


In [29]:
train_cluster_subspace, test_cluster_subspace = \
    add_k_means_n(X_train=X_train_scaled_imp_mv, X_test=X_test_scaled_imp_mv)
train_cluster_subspace.info()
test_cluster_subspace.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   k_means_2_n_0   86 non-null     bool 
 1   k_means_2_n_1   86 non-null     bool 
 2   k_means_3_n_0   86 non-null     bool 
 3   k_means_3_n_1   86 non-null     bool 
 4   k_means_3_n_2   86 non-null     bool 
 5   k_means_4_n_0   86 non-null     bool 
 6   k_means_4_n_1   86 non-null     bool 
 7   k_means_4_n_2   86 non-null     bool 
 8   k_means_4_n_3   86 non-null     bool 
 9   k_means_5_n_0   86 non-null     bool 
 10  k_means_5_n_1   86 non-null     bool 
 11  k_means_5_n_2   86 non-null     bool 
 12  k_means_5_n_3   86 non-null     bool 
 13  k_means_5_n_4   86 non-null     bool 
 14  k_means_6_n_0   86 non-null     bool 
 15  k_means_6_n_1   86 non-null     bool 
 16  k_means_6_n_2   86 non-null     bool 
 17  k_means_6_n_3   86 non-null     bool 
 18  k_means_6_n_4

In [30]:
X_train_scaled_imp_mv = pd.concat(objs=[X_train_scaled_imp_mv,
                                      train_cluster_subspace], axis=1)
X_test_scaled_imp_mv = pd.concat(objs=[X_test_scaled_imp_mv,
                                     test_cluster_subspace], axis=1)
X_train_scaled_imp_mv.info()
X_test_scaled_imp_mv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 141.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 61.0+ KB


In [31]:
k_means_feats_lst = list(train_cluster_subspace.columns)
k_means_feats_lst
features_list = features_list + k_means_feats_lst

['k_means_2_n_0',
 'k_means_2_n_1',
 'k_means_3_n_0',
 'k_means_3_n_1',
 'k_means_3_n_2',
 'k_means_4_n_0',
 'k_means_4_n_1',
 'k_means_4_n_2',
 'k_means_4_n_3',
 'k_means_5_n_0',
 'k_means_5_n_1',
 'k_means_5_n_2',
 'k_means_5_n_3',
 'k_means_5_n_4',
 'k_means_6_n_0',
 'k_means_6_n_1',
 'k_means_6_n_2',
 'k_means_6_n_3',
 'k_means_6_n_4',
 'k_means_6_n_5',
 'k_means_7_n_0',
 'k_means_7_n_1',
 'k_means_7_n_2',
 'k_means_7_n_3',
 'k_means_7_n_4',
 'k_means_7_n_5',
 'k_means_7_n_6',
 'k_means_8_n_0',
 'k_means_8_n_1',
 'k_means_8_n_2',
 'k_means_8_n_3',
 'k_means_8_n_4',
 'k_means_8_n_5',
 'k_means_8_n_6',
 'k_means_8_n_7',
 'k_means_9_n_0',
 'k_means_9_n_1',
 'k_means_9_n_2',
 'k_means_9_n_3',
 'k_means_9_n_4',
 'k_means_9_n_5',
 'k_means_9_n_6',
 'k_means_9_n_7',
 'k_means_9_n_8',
 'k_means_10_n_0',
 'k_means_10_n_1',
 'k_means_10_n_2',
 'k_means_10_n_3',
 'k_means_10_n_4',
 'k_means_10_n_5',
 'k_means_10_n_6',
 'k_means_10_n_7',
 'k_means_10_n_8',
 'k_means_10_n_9']

In [32]:
### reconstruct base line performance

base_perf_imp0_df = pd.DataFrame(columns=ordered_cols_lst)
base_perf_imp_med_df = pd.DataFrame(columns=ordered_cols_lst)
base_perf_imp_mv_df = pd.DataFrame(columns=ordered_cols_lst)

base_perfs_dict = {'imp0': base_perf_imp0_df,
                  'imp_med': base_perf_imp_med_df,
                  'imp_mv': base_perf_imp_mv_df}
imp_sets_dict = {'imp0': [X_train_scaled_imp0, X_test_scaled_imp0],
                 'imp_med': [X_train_scaled_imp_med, X_test_scaled_imp_med],
                 'imp_mv': [X_train_scaled_imp_mv, X_test_scaled_imp_mv]}
clf_dict = {'dt_clf': DecisionTreeClassifier, 'rf_clf': RandomForestClassifier,
            'ab_clf': AdaBoostClassifier, 'kn_clf': KNeighborsClassifier,
            'gnb_clf': GaussianNB, 'svc_clf': svm.SVC}

get_base_perfs(base_perfs_dict=base_perfs_dict,
                                 imp_sets_dict=imp_sets_dict,
                                 clf_dict=clf_dict, X_test=X_test,
                                 y_test=y_test)


 imp0

 dt_clf
DecisionTreeClassifier()
Training time: 0.007 s
Prediction time: 0.004 s
Confusion matrix:
 [[30  1]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.90909091, 0.75      ]), array([0.96774194, 0.5       ]), array([0.9375, 0.6   ]), array([31,  6], dtype=int64))

 rf_clf
RandomForestClassifier()
Training time: 0.091 s
Prediction time: 0.016 s
Confusion matrix:
 [[30  1]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.90909091, 0.75      ]), array([0.96774194, 0.5       ]), array([0.9375, 0.6   ]), array([31,  6], dtype=int64))

 ab_clf
AdaBoostClassifier()
Training time: 0.132 s
Prediction time: 0.004 s
Confusion matrix:
 [[31  0]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.91176471, 1.        ]), array([1. , 0.5]), array([0.95384615, 0.66666667]), array([31,  6], dtype=int64))

 kn_clf
KNeighborsClassifier()
Training time: 0.0 s
Prediction time: 0.0 s
Confusion matrix:
 [[28  3]
 [ 6  0]]
Precision, recall, f beta score, s

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AdaBoostClassifier()
Training time: 0.127 s
Prediction time: 0.016 s
Confusion matrix:
 [[30  1]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.90909091, 0.75      ]), array([0.96774194, 0.5       ]), array([0.9375, 0.6   ]), array([31,  6], dtype=int64))

 kn_clf
KNeighborsClassifier()
Training time: 0.0 s
Prediction time: 0.0 s
Confusion matrix:
 [[29  2]
 [ 5  1]]
Precision, recall, f beta score, support:
 (array([0.85294118, 0.33333333]), array([0.93548387, 0.16666667]), array([0.89230769, 0.22222222]), array([31,  6], dtype=int64))

 gnb_clf
GaussianNB()
Training time: 0.0 s
Prediction time: 0.0 s
Confusion matrix:
 [[28  3]
 [ 5  1]]
Precision, recall, f beta score, support:
 (array([0.84848485, 0.25      ]), array([0.90322581, 0.16666667]), array([0.875, 0.2  ]), array([31,  6], dtype=int64))

 svc_clf
SVC()
Training time: 0.016 s
Prediction time: 0.0 s
Confusion matrix:
 [[31  0]
 [ 6  0]]
Precision, recall, f beta score, support:
 (array([0.83783784, 0.        

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AdaBoostClassifier()
Training time: 0.147 s
Prediction time: 0.016 s
Confusion matrix:
 [[31  0]
 [ 3  3]]
Precision, recall, f beta score, support:
 (array([0.91176471, 1.        ]), array([1. , 0.5]), array([0.95384615, 0.66666667]), array([31,  6], dtype=int64))

 kn_clf
KNeighborsClassifier()
Training time: 0.016 s
Prediction time: 0.0 s
Confusion matrix:
 [[29  2]
 [ 6  0]]
Precision, recall, f beta score, support:
 (array([0.82857143, 0.        ]), array([0.93548387, 0.        ]), array([0.87878788, 0.        ]), array([31,  6], dtype=int64))

 gnb_clf
GaussianNB()
Training time: 0.016 s
Prediction time: 0.0 s
Confusion matrix:
 [[26  5]
 [ 4  2]]
Precision, recall, f beta score, support:
 (array([0.86666667, 0.28571429]), array([0.83870968, 0.33333333]), array([0.85245902, 0.30769231]), array([31,  6], dtype=int64))

 svc_clf
SVC()
Training time: 0.0 s
Prediction time: 0.018 s
Confusion matrix:
 [[31  0]
 [ 6  0]]
Precision, recall, f beta score, support:
 (array([0.83783784, 0.

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
base_perfs_dict['first_base'] = first_base_df
for key in key_order_lst:
    print('\n', key)
    base_perfs_dict[key]


 first_base


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.875000,0.40,0.903226,0.333333,0.888889,0.363636,31,6,28,3,4,2,0.000,0.000,DecisionTreeClassifier()
rf_clf,0.885714,1.00,1.000000,0.333333,0.939394,0.500000,31,6,31,0,4,2,0.096,0.003,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.875000,0.40,0.903226,0.333333,0.888889,0.363636,31,6,28,3,4,2,0.047,0.000,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.861111,1.00,1.000000,0.166667,0.925373,0.285714,31,6,31,0,5,1,0.000,0.000,KNeighborsClassifier()
gnb_clf,0.833333,0.16,0.322581,0.666667,0.465116,0.258065,31,6,10,21,2,4,0.000,0.000,GaussianNB()
svc_clf,0.837838,0.00,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.005,0.002,SVC()



 imp0


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.909091,0.750000,0.967742,0.500000,0.937500,0.600000,31,6,30,1,3,3,0.007,0.004,DecisionTreeClassifier()
rf_clf,0.909091,0.750000,0.967742,0.500000,0.937500,0.600000,31,6,30,1,3,3,0.091,0.016,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.911765,1.000000,1.000000,0.500000,0.953846,0.666667,31,6,31,0,3,3,0.132,0.004,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.823529,0.000000,0.903226,0.000000,0.861538,0.000000,31,6,28,3,6,0,0.000,0.000,KNeighborsClassifier()
gnb_clf,0.833333,0.142857,0.806452,0.166667,0.819672,0.153846,31,6,25,6,5,1,0.000,0.000,GaussianNB()
svc_clf,0.837838,0.000000,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.000,0.000,SVC()



 imp_med


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.843750,0.200000,0.870968,0.166667,0.857143,0.181818,31,6,27,4,5,1,0.016,0.000,DecisionTreeClassifier()
rf_clf,0.878788,0.500000,0.935484,0.333333,0.906250,0.400000,31,6,29,2,4,2,0.109,0.000,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.909091,0.750000,0.967742,0.500000,0.937500,0.600000,31,6,30,1,3,3,0.127,0.016,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.852941,0.333333,0.935484,0.166667,0.892308,0.222222,31,6,29,2,5,1,0.000,0.000,KNeighborsClassifier()
gnb_clf,0.848485,0.250000,0.903226,0.166667,0.875000,0.200000,31,6,28,3,5,1,0.000,0.000,GaussianNB()
svc_clf,0.837838,0.000000,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.016,0.000,SVC()



 imp_mv


,nonPOI_prec,POI_prec,nonPOI_rec,POI_rec,nonPOI_f,POI_f,nonPOI_sup,POI_sup,t_neg,f_neg,f_pos,t_pos,train_t,predict_t,model
dt_clf,0.843750,0.200000,0.870968,0.166667,0.857143,0.181818,31,6,27,4,5,1,0.016,0.000,DecisionTreeClassifier()
rf_clf,0.882353,0.666667,0.967742,0.333333,0.923077,0.444444,31,6,30,1,4,2,0.109,0.000,"(DecisionTreeClassifier(max_features='auto', r..."
ab_clf,0.911765,1.000000,1.000000,0.500000,0.953846,0.666667,31,6,31,0,3,3,0.147,0.016,"(DecisionTreeClassifier(max_depth=1, random_st..."
kn_clf,0.828571,0.000000,0.935484,0.000000,0.878788,0.000000,31,6,29,2,6,0,0.016,0.000,KNeighborsClassifier()
gnb_clf,0.866667,0.285714,0.838710,0.333333,0.852459,0.307692,31,6,26,5,4,2,0.016,0.000,GaussianNB()
svc_clf,0.837838,0.000000,1.000000,0.000000,0.911765,0.000000,31,6,31,0,6,0,0.000,0.018,SVC()


In [38]:
### Again, first, look for suspiciously highly significant features.
reg_imp0 = LinearRegression().fit(X=X_train_scaled_imp0, y=y_train)
reg_imp_med = LinearRegression().fit(X=X_train_scaled_imp_med, y=y_train)
reg_imp_mv = LinearRegression().fit(X=X_train_scaled_imp_mv, y=y_train)
coefs_df = pd.DataFrame(data={'reg_imp0.coef_': reg_imp0.coef_,
                              'reg_imp_med.coef_': reg_imp_med.coef_,
                              'reg_imp_mv.coef_': reg_imp_mv.coef_},
                        index=X_train_scaled_imp0.columns)
coefs_df

imp0_sort = np.sort(reg_imp0.coef_)
impmed_sort = np.sort(reg_imp_med.coef_)
impmv_sort = np.sort(reg_imp_mv.coef_)
high_idx = -10
low_idx = 9

print('imp0')
coefs_df.loc[(coefs_df['reg_imp0.coef_'] >= imp0_sort[high_idx])\
             | (coefs_df['reg_imp0.coef_'] <= imp0_sort[low_idx])]\
    ['reg_imp0.coef_'].sort_values()
print('imp_med')
coefs_df.loc[(coefs_df['reg_imp_med.coef_'] >= impmed_sort[high_idx])\
             | (coefs_df['reg_imp_med.coef_'] <= impmed_sort[low_idx])]\
    ['reg_imp_med.coef_'].sort_values()
print('imp_mv')
coefs_df.loc[(coefs_df['reg_imp_mv.coef_'] >= impmv_sort[high_idx])\
             | (coefs_df['reg_imp_mv.coef_'] <= impmv_sort[low_idx])]\
    ['reg_imp_mv.coef_'].sort_values()

,reg_imp0.coef_,reg_imp_med.coef_,reg_imp_mv.coef_
salary,-0.016663,0.025105,-0.057427
bonus,0.308671,0.138722,0.126613
long_term_incentive,-0.027691,0.053420,0.054426
expenses,0.035971,-0.001335,-0.098789
total_payments,-0.121670,-0.003680,-0.135331
...,...,...,...
k_means_10_n_5,0.003099,-0.003389,0.041383
k_means_10_n_6,-0.190877,-0.009233,-0.183810
k_means_10_n_7,0.212991,-0.198839,-0.089781
k_means_10_n_8,-0.239756,0.248528,0.207835


imp0


expenses_DivBy_salary_outliers                       -0.723459
to_messages_outliers                                 -0.375015
expenses_DivBy_bonus_outliers                        -0.330998
expenses_q_5                                         -0.304909
k_means_6_n_2                                        -0.304045
k_means_7_n_2                                        -0.304045
to_messages_DivBy_from_poi_to_this_person_outliers   -0.284732
long_term_incentive_DivBy_salary_outliers            -0.262513
restricted_stock_DivBy_exercised_stock_options_q_4   -0.262095
shared_receipt_with_poi_q_4                          -0.261221
total_payments_q_4                                    0.301059
bonus                                                 0.308671
bonus_DivBy_salary_outliers                           0.320587
restricted_stock_outliers                             0.345578
k_means_7_n_3                                         0.351098
expenses_DivBy_salary_q_5                             0

imp_med


expenses_DivBy_salary_outliers                      -0.584923
from_messages_outliers                              -0.369668
k_means_10_n_2                                      -0.368924
restricted_stock_DivBy_total_stock_value_outliers   -0.336320
salary_DivBy_bonus_q_5                              -0.278409
total_stock_value_outliers                          -0.269828
salary_DivBy_expenses_q_5                           -0.246531
exercised_stock_options_q_4                         -0.244164
salary_DivBy_long_term_incentive_q_10               -0.232841
exercised_stock_options_q_5                         -0.229259
expenses_DivBy_long_term_incentive_outliers          0.351064
salary_DivBy_bonus_outliers                          0.351792
salary_DivBy_expenses_q_4                            0.383538
exercised_stock_options                              0.403569
restricted_stock_outliers                            0.420542
long_term_incentive_outliers                         0.430664
total_st

imp_mv


to_messages_outliers                                             -0.411653
shared_receipt_with_poi_DivBy_from_poi_to_this_person_outliers   -0.396642
from_messages_outliers                                           -0.373504
expenses_DivBy_bonus_q_5                                         -0.317937
total_payments_DivBy_salary_q_4                                  -0.313774
k_means_4_n_2                                                    -0.311827
from_poi_to_this_person_DivBy_to_messages_outliers               -0.293301
total_stock_value_outliers                                       -0.282450
k_means_7_n_2                                                    -0.280980
from_messages                                                    -0.278495
restricted_stock_DivBy_total_stock_value_q_4                      0.321223
long_term_incentive_outliers                                      0.325740
total_payments_q_4                                                0.334968
from_messages_DivBy_from_

In [35]:
### Again, first, look for suspiciously highly significant features.
clf_imp0 = DecisionTreeClassifier().fit(X=X_train_scaled_imp0, y=y_train)
clf_imp_med = DecisionTreeClassifier().fit(X=X_train_scaled_imp_med, y=y_train)
clf_imp_mv = DecisionTreeClassifier().fit(X=X_train_scaled_imp_mv, y=y_train)
coefs_df = pd.DataFrame(data={
    'clf_imp0.feature_importances_': clf_imp0.feature_importances_,
    'clf_imp_med.feature_importances_': clf_imp_med.feature_importances_,
    'clf_imp_mv.feature_importances_': clf_imp_mv.feature_importances_
},
                        index=X_train_scaled_imp0.columns)

coefs_df.loc[(coefs_df['clf_imp0.feature_importances_'] > 0)]\
    ['clf_imp0.feature_importances_'].sort_values(ascending=False)
coefs_df.loc[(coefs_df['clf_imp_med.feature_importances_'] > 0)]\
    ['clf_imp_med.feature_importances_'].sort_values(ascending=False)
coefs_df.loc[(coefs_df['clf_imp_mv.feature_importances_'] > 0)]\
['clf_imp_mv.feature_importances_'].sort_values(ascending=False)

shared_receipt_with_poi_DivBy_to_messages         0.183665
exercised_stock_options                           0.146825
salary_DivBy_long_term_incentive_q_4              0.137738
bonus                                             0.124554
total_payments_DivBy_expenses                     0.116216
expenses_DivBy_salary                             0.077477
restricted_stock_DivBy_exercised_stock_options    0.073788
long_term_incentive_DivBy_expenses_q_5            0.072635
restricted_stock_outliers                         0.044427
expenses                                          0.022674
Name: clf_imp0.feature_importances_, dtype: float64

exercised_stock_options                            0.146825
total_payments_DivBy_long_term_incentive           0.137738
bonus                                              0.124554
to_messages_DivBy_shared_receipt_with_poi          0.119100
total_stock_value_DivBy_exercised_stock_options    0.077477
salary_DivBy_total_payments_q_4                    0.073788
expenses_DivBy_salary_q_5                          0.072635
expenses_DivBy_total_payments                      0.064565
to_messages_outliers                               0.064565
salary_DivBy_bonus_q_10                            0.051652
restricted_stock_outliers                          0.044427
total_payments_DivBy_salary                        0.022674
Name: clf_imp_med.feature_importances_, dtype: float64

from_messages_DivBy_from_this_person_to_poi                   0.214755
shared_receipt_with_poi_q_5                                   0.214703
shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_10    0.154955
salary_DivBy_long_term_incentive                              0.130743
shared_receipt_with_poi_DivBy_to_messages                     0.121059
long_term_incentive_DivBy_salary_q_4                          0.091150
bonus                                                         0.072635
Name: clf_imp_mv.feature_importances_, dtype: float64

### Feature selection

I have a suspicion that many of these most highest- and lowest-coefficient features would end up overfitting a linear regression model. They aren't very plausible and may just be picking up on outliers.

Luckily, I'm not doing a regression but a classification problem. And, maybe more of the decision tree important features seem plausible.

That said, it may still be an indication that I may need to prune my ratio features (and their dependent features) to include only plausible ratios.

Okay, I pruned the most implausible features (ratios across subspaces).

In [39]:
list(X_train.count().loc[X_train.count() < 20].index)

[]

In [43]:
X_train_scaled_imp0.columns[280:]

Index(['from_messages_DivBy_from_this_person_to_poi_sign',
       'from_this_person_to_poi_DivBy_from_messages_sign', 'k_means_2_n_0',
       'k_means_2_n_1', 'k_means_3_n_0', 'k_means_3_n_1', 'k_means_3_n_2',
       'k_means_4_n_0', 'k_means_4_n_1', 'k_means_4_n_2', 'k_means_4_n_3',
       'k_means_5_n_0', 'k_means_5_n_1', 'k_means_5_n_2', 'k_means_5_n_3',
       'k_means_5_n_4', 'k_means_6_n_0', 'k_means_6_n_1', 'k_means_6_n_2',
       'k_means_6_n_3', 'k_means_6_n_4', 'k_means_6_n_5', 'k_means_7_n_0',
       'k_means_7_n_1', 'k_means_7_n_2', 'k_means_7_n_3', 'k_means_7_n_4',
       'k_means_7_n_5', 'k_means_7_n_6', 'k_means_8_n_0', 'k_means_8_n_1',
       'k_means_8_n_2', 'k_means_8_n_3', 'k_means_8_n_4', 'k_means_8_n_5',
       'k_means_8_n_6', 'k_means_8_n_7', 'k_means_9_n_0', 'k_means_9_n_1',
       'k_means_9_n_2', 'k_means_9_n_3', 'k_means_9_n_4', 'k_means_9_n_5',
       'k_means_9_n_6', 'k_means_9_n_7', 'k_means_9_n_8', 'k_means_10_n_0',
       'k_means_10_n_1', 'k_means_10_n_

In [44]:
with open('X_train.pkl', 'wb') as file:
    pickle.dump(obj=X_train, file=file)
with open('y_train.pkl', 'wb') as file:
    pickle.dump(obj=y_train, file=file)
with open('X_test.pkl', 'wb') as file:
    pickle.dump(obj=X_test, file=file)
with open('y_test.pkl', 'wb') as file:
    pickle.dump(obj=y_test, file=file)
with open('X_train_scaled_imp0.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp0, file=file)
with open('X_test_scaled_imp0.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp0, file=file)
with open('X_train_scaled_imp_med.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp_med, file=file)
with open('X_test_scaled_imp_med.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp_med, file=file)
with open('X_train_scaled_imp_mv.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp_mv, file=file)
with open('X_test_scaled_imp_mv.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp_mv, file=file)
    
with open('fin_features.pkl', 'wb') as file:
    pickle.dump(obj=fin_features, file=file)
with open('pay_features.pkl', 'wb') as file:
    pickle.dump(obj=pay_features, file=file)
with open('stock_features.pkl', 'wb') as file:
    pickle.dump(obj=stock_features, file=file)
with open('email_features.pkl', 'wb') as file:
    pickle.dump(obj=email_features, file=file)
with open('pay_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=pay_feats_divby_lst, file=file)
with open('stock_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=stock_feats_divby_lst, file=file)
with open('email_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=email_feats_divby_lst, file=file)
with open('quant_flags_lst.pkl', 'wb') as file:
    pickle.dump(obj=quant_flags_lst, file=file)
with open('sign_flags_lst.pkl', 'wb') as file:
    pickle.dump(obj=sign_flags_lst, file=file)
features_list = ['poi'] + fin_features + email_features + pay_feats_divby_lst \
    + stock_feats_divby_lst + email_feats_divby_lst + quant_flags_lst \
    + sign_flags_lst
with open('features_list.pkl', 'wb') as file:
    pickle.dump(obj=features_list, file=file)
    
with open('base_perfs_dict.pkl', 'wb') as file:
    pickle.dump(obj=base_perfs_dict, file=file)

In [58]:
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

NameError: name 'data_dict' is not defined

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)